## Data Exploration / Cleaning

In [1]:
import pandas as pd

# ============================
# 1. Load datasets
# ============================
train_df = pd.read_csv(r'F:\AI Projects\Regression Project\data\raw\train_data.csv')
eval_df = pd.read_csv(r'F:\AI Projects\Regression Project\data\raw\eval_data.csv')
metros = pd.read_csv(r'F:\AI Projects\Regression Project\data\raw\usmetros.csv')

pd.set_option('display.max_columns', None)  # to display all columns
pd.set_option('display.max_rows', None)     # to display all rows


In [2]:
print(train_df.shape)
print(eval_df.shape)

(585244, 39)
(149424, 39)


In [3]:
train_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-03-31,200000.0,7500.0,104.931794,79.265873,1.0,1.0,1.0,2.0,290.0,0.909091,0.000000,0.000000,PGH,15469,2012,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2441.0,41.8,20241.0,2385.0,1108.0,641.0,94600.0,1171.0,52.0,2376.0,2376.0,1018.0,105863.681174,Pittsburgh


In [4]:
metros.head(2)

,metro_fips,metro,metro_ascii,metro_full,county_name,county_fips,state_id,state_name,lat,lng,population
0,35620,New York,New York,"New York-Newark-Jersey City, NY-NJ",Suffolk,36103,NY,New York,40.7222,-74.0225,19498249
1,31080,Los Angeles,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,6037,CA,California,34.2215,-118.1494,12799100


In [5]:
train_df['city_full'].value_counts().head()

city_full
New York-Newark-Jersey City       78020
Chicago-Naperville-Elgin          35344
Los Angeles-Long Beach-Anaheim    33840
Philadelphia-Camden-Wilmington    31396
DC_Metro                          29516
Name: count, dtype: int64

### Map cities to Lat/Long
- The goal is to use Lattitude and longitude instead of cities for our ML models

In [6]:
# ============================
# 2. Fix city name mismatches
# ============================
city_mapping = {
    'Las Vegas-Henderson-Paradise': 'Las Vegas-Henderson-North Las Vegas',
    'Denver-Aurora-Lakewood': 'Denver-Aurora-Centennial',
    'Houston-The Woodlands-Sugar Land': 'Houston-Pasadena-The Woodlands',
    'Austin-Round Rock-Georgetown': 'Austin-Round Rock-San Marcos',
    'Miami-Fort Lauderdale-Pompano Beach': 'Miami-Fort Lauderdale-West Palm Beach',
    'San Francisco-Oakland-Berkeley': 'San Francisco-Oakland-Fremont',
    'DC_Metro': 'Washington-Arlington-Alexandria',
    'Atlanta-Sandy Springs-Alpharetta': 'Atlanta-Sandy Springs-Roswell'
}

In [7]:
def merge_city_lat_lng(df, metros):
    df = df.copy()
    metros = metros.copy()

    # Drop existing lat/lng
    df = df.drop(columns=["lat", "lng"], errors="ignore")

    # Normalize helper
    def normalize(s):
        return (
            str(s)
            .lower()
            .replace("-", " ")
            .replace("_", " ")
            .replace(".", "")
            .strip()
        )

    def first_word(s):
        return normalize(s).split()[0]

    # Create keys
    df["city_key"] = df["city_full"].apply(first_word)
    metros["metro_key"] = metros["metro_full"].apply(first_word)

    # DC override (required)
    df["city_key"] = df["city_key"].replace({"dc": "washington"})

    # ---- FIRST MERGE (fast path) ----
    df = df.merge(
        metros[["metro_key", "lat", "lng"]],
        how="left",
        left_on="city_key",
        right_on="metro_key"
    )

    df.drop(columns=["metro_key"], inplace=True)

    # ---- FALLBACK FOR REMAINING ROWS ----
    missing_mask = df["lat"].isna()

    if missing_mask.any():
        metro_lookup = metros.copy()
        metro_lookup["metro_norm"] = metro_lookup["metro_full"].apply(normalize)

        for idx in df[missing_mask].index:
            city = normalize(df.at[idx, "city_full"])

            match = metro_lookup[
                metro_lookup["metro_norm"].str.contains(city.split()[0])
            ]

            if not match.empty:
                df.at[idx, "lat"] = match.iloc[0]["lat"]
                df.at[idx, "lng"] = match.iloc[0]["lng"]

    # Final log
    still_missing = df.loc[df["lat"].isna(), "city_full"].unique()
    if len(still_missing) > 0:
        print(f"⚠️ Still missing lat/lng for: {list(still_missing)}")

    return df


In [8]:
train_df = merge_city_lat_lng(train_df, metros)
eval_df  = merge_city_lat_lng(eval_df, metros)


In [9]:
train_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full,city_key,lat,lng
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta,atlanta,33.7338,-84.3922
1,2012-03-31,200000.0,7500.0,104.931794,79.265873,1.0,1.0,1.0,2.0,290.0,0.909091,0.000000,0.000000,PGH,15469,2012,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2441.0,41.8,20241.0,2385.0,1108.0,641.0,94600.0,1171.0,52.0,2376.0,2376.0,1018.0,105863.681174,Pittsburgh,pittsburgh,40.4744,-79.8632


In [10]:
train_df.isnull().sum()

date                            0
median_sale_price               0
median_list_price               0
median_ppsf                     0
median_list_ppsf                0
homes_sold                      0
pending_sales                   0
new_listings                    0
inventory                       0
median_dom                      0
avg_sale_to_list                0
sold_above_list                 0
off_market_in_two_weeks         0
city                            0
zipcode                         0
year                            0
bank                            0
bus                             0
hospital                        0
mall                            0
park                            0
restaurant                      0
school                          0
station                         0
supermarket                     0
Total Population                0
Median Age                      0
Per Capita Income               0
Total Families Below Poverty    0
Total Housing 

In [11]:
eval_df.isnull().sum()

date                            0
median_sale_price               0
median_list_price               0
median_ppsf                     0
median_list_ppsf                0
homes_sold                      0
pending_sales                   0
new_listings                    0
inventory                       0
median_dom                      0
avg_sale_to_list                0
sold_above_list                 0
off_market_in_two_weeks         0
city                            0
zipcode                         0
year                            0
bank                            0
bus                             0
hospital                        0
mall                            0
park                            0
restaurant                      0
school                          0
station                         0
supermarket                     0
Total Population                0
Median Age                      0
Per Capita Income               0
Total Families Below Poverty    0
Total Housing 

In [12]:
print(train_df.shape)
print(eval_df.shape)

(989914, 42)
(252744, 42)


## clean Duplicates

In [13]:
print(train_df.shape)


duplicated_rows = train_df[train_df.duplicated()].shape[0]
print("duplicated_rows:", duplicated_rows)

duplicated_rows = train_df[train_df.duplicated(subset=train_df.columns.difference(['date', 'year']))].shape[0]
print("duplicated_rows excluding date column:", duplicated_rows)

(989914, 42)
duplicated_rows: 0
duplicated_rows excluding date column: 11958


In [14]:
# Delete duplicates
train_df = train_df.drop_duplicates(subset=train_df.columns.difference(['date', 'year']), keep=False)

print(train_df.shape)

duplicated_rows = train_df[train_df.duplicated()].shape[0]
print("duplicated_rows:", duplicated_rows)

duplicated_rows = train_df[train_df.duplicated(subset=train_df.columns.difference(['date', 'year']))].shape[0]
print("duplicated_rows excluding date column:", duplicated_rows)

(974149, 42)
duplicated_rows: 0
duplicated_rows excluding date column: 0


In [15]:
print(eval_df.shape)


duplicated_rows = eval_df[eval_df.duplicated()].shape[0]
print("duplicated_rows:", duplicated_rows)

duplicated_rows = eval_df[eval_df.duplicated(subset=eval_df.columns.difference(['date', 'year']))].shape[0]
print("duplicated_rows excluding date column:", duplicated_rows)

(252744, 42)
duplicated_rows: 0
duplicated_rows excluding date column: 1317


In [16]:
# Delete duplicates
eval_df = eval_df.drop_duplicates(subset=eval_df.columns.difference(['date', 'year']), keep=False)

print(eval_df.shape)


duplicated_rows = eval_df[eval_df.duplicated()].shape[0]
print("duplicated_rows:", duplicated_rows)

duplicated_rows = eval_df[eval_df.duplicated(subset=eval_df.columns.difference(['date', 'year']))].shape[0]
print("duplicated_rows excluding date column:", duplicated_rows)

(250985, 42)
duplicated_rows: 0
duplicated_rows excluding date column: 0


## clean outliers

In [17]:
train_df['median_list_price'].describe()

count    9.741490e+05
mean     4.460819e+05
std      1.831605e+06
min      0.000000e+00
25%      1.898980e+05
50%      3.220000e+05
75%      5.290000e+05
max      1.000000e+09
Name: median_list_price, dtype: float64

In [27]:
import plotly.io as pio
pio.renderers.default = "browser"


In [28]:
import plotly.express as px

fig = px.violin(
    train_df,
    y="median_list_price",
    box=True,
    hover_name="median_list_price"
)

fig.update_layout(title="Violin Plot of Median List Price")
fig.show()


In [29]:
top_1_percent = train_df.nlargest(int(0.01 * len(train_df)), 'median_list_price')
print(top_1_percent.shape)
top_1_percent.head(10)

(9741, 42)


,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full,city_key,lat,lng
459580,2015-10-31,439000.0,999999999.0,218.867925,236.391156,243.0,269.0,671.0,259.0,46.0,0.980975,0.160494,0.130112,DC,20878,2015,9.0,0.0,4.0,1.0,110.0,59.0,50.0,2.0,11.0,63855.0,38.1,52955.0,63757.0,23514.0,1677.0,535500.0,36595.0,1960.0,61237.0,61237.0,32125.0,5.143890e+05,DC_Metro,washington,38.8446,-77.5092
466659,2015-11-30,425000.0,999999999.0,220.055710,236.634569,241.0,284.0,616.0,196.0,47.0,0.979123,0.132780,0.133803,DC,20878,2015,9.0,0.0,4.0,1.0,110.0,59.0,50.0,2.0,11.0,63855.0,38.1,52955.0,63757.0,23514.0,1677.0,535500.0,36595.0,1960.0,61237.0,61237.0,32125.0,5.148844e+05,DC_Metro,washington,38.8446,-77.5092
479674,2015-12-31,439444.0,999999999.0,217.479377,231.798505,254.0,255.0,536.0,136.0,53.0,0.976354,0.137795,0.109804,DC,20878,2015,9.0,0.0,4.0,1.0,110.0,59.0,50.0,2.0,11.0,63855.0,38.1,52955.0,63757.0,23514.0,1677.0,535500.0,36595.0,1960.0,61237.0,61237.0,32125.0,5.152060e+05,DC_Metro,washington,38.8446,-77.5092
528325,2016-05-31,270000.0,50449949.5,232.758621,4156.568441,1.0,1.0,2.0,2.0,264.0,1.000000,0.000000,0.000000,DC,20838,2016,0.0,0.0,0.0,0.0,11.0,1.0,4.0,2.0,0.0,247.0,49.6,61165.0,243.0,93.0,1507.0,607100.0,135.0,0.0,239.0,239.0,122.0,6.739973e+05,DC_Metro,washington,38.8446,-77.5092
547119,2016-06-30,415000.0,50449949.5,257.508343,4156.568441,2.0,2.0,2.0,2.0,324.5,1.000447,0.500000,0.000000,DC,20838,2016,0.0,0.0,0.0,0.0,11.0,1.0,4.0,2.0,0.0,247.0,49.6,61165.0,243.0,93.0,1507.0,607100.0,135.0,0.0,239.0,239.0,122.0,6.739973e+05,DC_Metro,washington,38.8446,-77.5092
556206,2016-07-31,270000.0,50449949.5,232.758621,4156.568441,3.0,3.0,2.0,2.0,264.0,0.973591,0.333333,0.000000,DC,20838,2016,0.0,0.0,0.0,0.0,11.0,1.0,4.0,2.0,0.0,247.0,49.6,61165.0,243.0,93.0,1507.0,607100.0,135.0,0.0,239.0,239.0,122.0,6.739973e+05,DC_Metro,washington,38.8446,-77.5092
481345,2015-12-31,830000.0,27800000.0,459.833795,3677.735150,3.0,3.0,1.0,22.0,235.0,0.842625,0.000000,0.000000,NY,7620,2015,10.0,0.0,10.0,0.0,46.0,18.0,69.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.366853e+06,New York-Newark-Jersey City,new,40.7222,-74.0225
481346,2015-12-31,830000.0,27800000.0,459.833795,3677.735150,3.0,3.0,1.0,22.0,235.0,0.842625,0.000000,0.000000,NY,7620,2015,10.0,0.0,10.0,0.0,46.0,18.0,69.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.366853e+06,New York-Newark-Jersey City,new,29.7832,-89.9570
481347,2015-12-31,830000.0,27800000.0,459.833795,3677.735150,3.0,3.0,1.0,22.0,235.0,0.842625,0.000000,0.000000,NY,7620,2015,10.0,0.0,10.0,0.0,46.0,18.0,69.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.366853e+06,New York-Newark-Jersey City,new,41.3613,-72.8445
509346,2016-03-31,4050000.0,19990000.0,613.636364,2815.492958,2.0,2.0,1.0,4.0,183.0,0.942957,0.000000,0.000000,NY,11962,2016,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,415.0,46.9,99829.0,415.0,838.0,1440.0,2000001.0,167.0,0.0,415.0,415.0,128.0,3.632048e+06,New York-Newark-Jersey City,new,40.7222,-74.0225


In [30]:
top_1_percent = train_df.nlargest(int(0.01 * len(train_df)), 'median_list_price')
print(top_1_percent['median_list_price'].value_counts().sort_index(ascending=False))

median_list_price
999999999.0      3
50449949.5       3
27800000.0       3
19990000.0     114
15849000.0       3
15624500.0       9
15500000.0       3
13995000.0       3
13100000.0      12
12772000.0       3
12500000.0      15
11900000.0       3
11495000.0       6
10747500.0       6
10692000.0      16
10000000.0       3
9999998.0        1
9934000.0        2
9697500.0        9
9547500.0        6
9394000.0        1
9350000.0        3
9100000.0        6
8900000.0        6
8793944.0       15
8750000.0        3
8700000.0       12
8500000.0        1
8495000.0        6
8142500.0        6
8000000.0        3
7995000.0        3
7988000.0        6
7947500.0        6
7941500.0        6
7750000.0       12
7695000.0        6
7522500.0        3
7495000.0        6
7472500.0        3
7450000.0        6
7442500.0        2
7424000.0        6
7374950.0        1
7300000.0        9
7299500.0        1
7245000.0        6
7199000.0       15
7000000.0        3
6999500.0        3
6995000.0      186
6945000.0    

- Investigate if median_list_price outliers are independent in distinct regions (for example if DC has different median_list_price)
- Drop outliers to keep things realistic and clean

In [31]:
# Clean outliers above 19M in both train and eval
train_df = train_df[train_df['median_list_price'] <= 19_000_000].copy()
eval_df = eval_df[eval_df['median_list_price'] <= 19_000_000].copy()

In [32]:
import plotly.express as px

fig = px.violin(train_df, y="median_list_price", box=True, hover_name="median_list_price")
fig.update_layout(title="Violin Plot of Median List Price")
fig.show()

In [33]:
top_1_percent = train_df.nlargest(int(0.01 * len(train_df)), 'median_list_price')
print(top_1_percent['median_list_price'].value_counts().sort_index(ascending=False))

median_list_price
15849000.0      3
15624500.0      9
15500000.0      3
13995000.0      3
13100000.0     12
12772000.0      3
12500000.0     15
11900000.0      3
11495000.0      6
10747500.0      6
10692000.0     16
10000000.0      3
9999998.0       1
9934000.0       2
9697500.0       9
9547500.0       6
9394000.0       1
9350000.0       3
9100000.0       6
8900000.0       6
8793944.0      15
8750000.0       3
8700000.0      12
8500000.0       1
8495000.0       6
8142500.0       6
8000000.0       3
7995000.0       3
7988000.0       6
7947500.0       6
7941500.0       6
7750000.0      12
7695000.0       6
7522500.0       3
7495000.0       6
7472500.0       3
7450000.0       6
7442500.0       2
7424000.0       6
7374950.0       1
7300000.0       9
7299500.0       1
7245000.0       6
7199000.0      15
7000000.0       3
6999500.0       3
6995000.0     186
6945000.0       3
6847500.0       6
6844000.0      12
6822000.0       6
6720000.0       3
6700000.0       6
6665000.0       6
6650000.0 

In [34]:
# ============================
# 4. Save cleaned datasets
# ============================
train_df.to_csv(
    r"F:\AI Projects\Regression Project\data\processed\cleaning_train.csv",
    index=False
)

eval_df.to_csv(
    r"F:\AI Projects\Regression Project\data\processed\cleaning_eval.csv",
    index=False
)


print("✅ Cleaning complete.")

✅ Cleaning complete.
